In [1]:
import msgspec
import cloudpickle as pickle
import typing

In [2]:
def create_struct(function: typing.Callable) -> msgspec.Struct:

    # Get standard type hints
    msg_spec_hints = typing.get_type_hints(function)

    assert "return" in msg_spec_hints and len(msg_spec_hints) > 1, \
        "Typehint must be specified for all arguments and return value."

    msg_spec_hints = list(msg_spec_hints.items())

    # Handle `return` type hint
    msg_spec_hints[-1] = tuple([msg_spec_hints[-1][0], typing.Optional[msg_spec_hints[-1][1]], None])

    # Create struct
    Function = msgspec.defstruct(
        "Function",
        msg_spec_hints,
    )
    print(type(Function))

    return Function

def enc_hook(obj: typing.Any) -> bytes:
    return pickle.dumps(obj)

def dec_hook(type: typing.Any, obj: bytes) -> typing.Any:
    return pickle.loads(obj)

In [3]:
class TestClass:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

def test_function(a: int, b: str, c: typing.List[int], d: TestClass) -> TestClass:
    return TestClass(a, b, c)

MyStruct = create_struct(test_function)

encoder = msgspec.msgpack.Encoder(enc_hook=enc_hook)
decoder = msgspec.msgpack.Decoder(type=MyStruct, dec_hook=dec_hook)

my_struct = MyStruct(1, "2", [3, 4], TestClass(5, "6", [7, 8]))
print(my_struct)

<class 'msgspec._core.StructMeta'>
Function(a=1, b='2', c=[3, 4], d=<__main__.TestClass object at 0x10584f7d0>, return=None)


In [4]:
encoded = encoder.encode(my_struct)
print(type(encoded))
print(encoded)

<class 'bytes'>
b'\x85\xa1a\x01\xa1b\xa12\xa1c\x92\x03\x04\xa1d\xc5\x03"\x80\x05\x95\x17\x03\x00\x00\x00\x00\x00\x00\x8c\x17cloudpickle.cloudpickle\x94\x8c\x14_make_skeleton_class\x94\x93\x94(\x8c\x08builtins\x94\x8c\x04type\x94\x93\x94\x8c\tTestClass\x94h\x03\x8c\x06object\x94\x93\x94\x85\x94}\x94\x8c\n__module__\x94\x8c\x08__main__\x94s\x8c 232cfd4b8e2b4473ad8316d89f18141c\x94Nt\x94R\x94h\x00\x8c\x0f_class_setstate\x94\x93\x94h\x0f}\x94(h\x0bh\x0c\x8c\x08__init__\x94h\x00\x8c\x0e_make_function\x94\x93\x94(h\x00\x8c\r_builtin_type\x94\x93\x94\x8c\x08CodeType\x94\x85\x94R\x94(K\x04K\x00K\x00K\x04K\x02K\x03C0\x97\x00|\x01|\x00_\x00\x00\x00\x00\x00\x00\x00\x00\x00|\x02|\x00_\x01\x00\x00\x00\x00\x00\x00\x00\x00|\x03|\x00_\x02\x00\x00\x00\x00\x00\x00\x00\x00d\x00S\x00\x94N\x85\x94\x8c\x01a\x94\x8c\x01b\x94\x8c\x01c\x94\x87\x94(\x8c\x04self\x94h\x1dh\x1eh\x1ft\x94\x8cN/var/folders/1z/64_91wwj46ng1xjffddgz_n40000gn/T/ipykernel_51892/2604510142.py\x94h\x13\x8c\x12TestClass.__init__\x94K\x02C\

In [5]:
decoded = decoder.decode(encoded)
print(decoded)

Function(a=1, b='2', c=[3, 4], d=<__main__.TestClass object at 0x105a5c990>, return=None)


In [7]:
print(decoded.d.a)

5
